## GRV API

- acc email: maxharper26@icloud.com
- acc password: kowGot-sonbyk-4fazko
- api key: 4157732f-fa7f-4459-ab26-089e4deab17a
- documentation: https://topaz.grv.org.au/docs/#/codes/GetTrack

In [1]:
import pandas as pd
from topaz import TopazAPI
import time
import requests
from requests.exceptions import HTTPError
from json import JSONDecodeError
import numpy as np

In [2]:
key = '4157732f-fa7f-4459-ab26-089e4deab17a'
key_m = '6a8ffdc7-ee7a-4795-9b98-607e3e838342'
key_p ='34b33827-3ca1-4c2b-a4c5-40fde1d99a84'


In [3]:
topaz_api = TopazAPI(key)

In [4]:
authority_codes = [ 'NSW', 'QLD', 'SA', 'TAS', 'VIC', 'WA','NT']
all_races = []
missed_run=False

for code in authority_codes:
    try:
        races = topaz_api.get_races(from_date='2024-03-04', to_date='2024-03-08', owning_authority_code=code)
        all_races.append(races)
    except Exception as e:
        print(f"Error fetching races for {code}: {e}")

all_races_df = pd.concat(all_races, ignore_index=True)
race_ids = list(all_races_df['raceId'].unique())
race_ids

[1002606719,
 1002606720,
 1002606721,
 1002606722,
 1002606723,
 1002606724,
 1002606725,
 1002606726,
 1002606727,
 1002606728,
 1002606729,
 1002606730,
 1002640315,
 1002640316,
 1002640317,
 1002640318,
 1002640319,
 1002640320,
 1002640321,
 1002640322,
 1002640323,
 1002640324,
 1002682362,
 1002682363,
 1002682364,
 1002682365,
 1002682366,
 1002682367,
 1002682368,
 1002682369,
 1002682370,
 1002682371,
 1002682372,
 1002682373,
 1003114224,
 1003114225,
 1003114226,
 1003114227,
 1003114228,
 1003114229,
 1003114230,
 1003114231,
 1003114232,
 1003114233,
 1003117731,
 1003117732,
 1003117733,
 1003117734,
 1003117735,
 1003117736,
 1003117737,
 1003117738,
 1003117739,
 1003117740,
 1003117741,
 1003117742,
 1003117743,
 1003117744,
 1003117745,
 1003117746,
 1003117747,
 1003117748,
 1003117749,
 1003117750,
 1003117751,
 1003117752,
 1003117753,
 1003117754,
 1003117755,
 1003117756,
 1003117757,
 1003117758,
 1003117759,
 1003117760,
 1003117761,
 1003117762,
 1003948389,

In [5]:
len(race_ids)

713

In [6]:
# race_ids = pd.read_csv('api_files/missing.csv')['missed']
# missed_run=True
# race_ids

In [7]:
big = ''
start = time.time()
first=0
headers={'X-API-Key':key}
missed=[]



for i in range(first,len(race_ids)):
    rID = race_ids[i]
    url = f'https://topaz.grv.org.au/api/race/{rID}/result'
    try:
        race = requests.get(url.format(rID),headers=headers).json()
    except JSONDecodeError as e:
        race={'error':1,'message':'empty json'}
    
    if('error' in race and 'message' in race):
        print(rID,race['message'])
        missed.append(rID)
    else:
        splits = race['splitTimes']
        
        
        summary = pd.DataFrame(race['runs'])

        cols=['track','distance','raceId','weightInKg','meetingDate','dogName','resultTime','bestFinishTrackAndDistance', 'pir','incomingGrade', 'outgoingGrade', 'rating','resultMarginLengths', 'startPaceCode', 'jumpCode',
           'startPrice','boxNumber']
        summary=summary[cols]
        summary['Plc'] = summary['resultTime'].rank(method='min')

        if(splits!=[]):
            splits = pd.DataFrame(splits)
            splits = splits[['dogName','splitTimeMarker','time']]

            markers = max(splits['splitTimeMarker'])
            for marker in range(1,markers+1):
                split2 = splits[splits['splitTimeMarker']==marker]
                split2=split2.rename(columns={'time':'split_'+str(marker)})[['dogName','split_'+str(marker)]]
                summary = pd.merge(summary,split2,on='dogName',how='outer')


        if(type(big)==str):
            big = summary
        else:
            big = pd.concat([big,summary])

            



end = time.time()
taken = end-start
print('time for',len(race_ids),':',round(taken,3),'avg:',round(taken/len(race_ids),2))


time for 713 : 2359.949 avg: 3.31


In [8]:
big

,track,distance,raceId,weightInKg,meetingDate,dogName,resultTime,bestFinishTrackAndDistance,pir,incomingGrade,...,rating,resultMarginLengths,startPaceCode,jumpCode,startPrice,boxNumber,Plc,split_1,split_2,split_3
0,Richmond,320,1002606719,28.1,2024-03-04T00:00:00.000Z,SWIFT GETAWAY,18.78,18.78,331,Maiden,...,0,1.25L,None,None,5.0,1.0,1.0,2.23,7.87,NaN
1,Richmond,320,1002606719,27.2,2024-03-04T00:00:00.000Z,GALLIPOLI GAL,19.12,19.00,556,Maiden,...,90,5.00L,None,None,3.6,2.0,6.0,2.29,8.12,NaN
2,Richmond,320,1002606719,23.7,2024-03-04T00:00:00.000Z,WHAT KEPT YOU,19.42,19.42,218,Maiden,...,0,9.75L,None,None,8.0,3.0,8.0,2.21,8.27,NaN
3,Richmond,320,1002606719,28.1,2024-03-04T00:00:00.000Z,RUSSO,19.10,19.03,444,Maiden,...,0,4.75L,None,None,6.0,4.0,5.0,2.27,8.01,NaN
4,Richmond,320,1002606719,29.0,2024-03-04T00:00:00.000Z,BUTKER,18.86,18.86,662,Maiden,...,0,1.25L,None,None,6.0,5.0,3.0,2.31,7.87,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,Darwin,312,1003936393,30.8,2024-03-06T00:00:00.000Z,NORTHERN EXPRESS,18.49,18.35,None,5,...,94,3.50L,None,None,6.7,1.0,4.0,NaN,NaN,NaN
1,Darwin,312,1003936393,23.4,2024-03-06T00:00:00.000Z,NOT ALL THERE,18.76,18.39,None,5,...,90,7.50L,None,None,8.6,2.0,5.0,NaN,NaN,NaN
2,Darwin,312,1003936393,25.7,2024-03-06T00:00:00.000Z,SPITEFUL MISS,18.26,18.15,0,4,...,90,0.75L,None,None,3.9,4.0,1.0,7.20,11.06,NaN
3,Darwin,312,1003936393,33.6,2024-03-06T00:00:00.000Z,SOLSBURY HILL,18.47,18.14,0,5,...,92,3.25L,None,None,5.3,6.0,3.0,NaN,NaN,NaN


In [9]:
big1 = big.copy()
big1['Track']=big1['track'].str.lower()
big1['dogName']=big1['dogName'].str.lower()

to_rename={
    'Plc':'Place',
    'dogName':'DogName',
    'boxNumber':'Box',
    'resultTime':'RunTime',
    'weightInKg':'Weight',
    'startPrice':'StartPrice',
    'pir':'PIR',
    'distance':'Distance',
    'raceId':'RaceId',
    'incomingGrade':'RaceGrade',
    'meetingDate':'date',
    'split_1':'Split'
}

big1=big1.rename(columns=to_rename)
big1 = big1[['Place','DogName','Box','Weight','StartPrice','PIR','Split','RunTime','RaceId','Distance','RaceGrade','Track','date']]
big1['TrainerLName']='poo'
big1['TrainerFName']='poo'

big1['date']=big1['date'].astype(str).str.split('T').str[0]
big1['date']=pd.to_datetime(big1['date'])


import os
files = os.listdir('api_files')
files = [x for x in files if 'missing' not in x and '.DS' not in x]
m=0
print(files)
for file in files:
    num = file.split('_')[1].split('.')[0]
    num=int(num)
    if(num>m):
        m=num
print(m)

big1.to_csv('api_files/api_'+str(m+1)+'.csv',index=False)
big1

['api_8.csv', 'api_9.csv', 'api_17.csv', 'api_16.csv', 'api_14.csv', 'api_28.csv', 'api_29.csv', 'api_15.csv', 'api_11.csv', 'api_10.csv', 'api_12.csv', 'api_13.csv', 'api_22.csv', 'api_23.csv', 'api_21.csv', 'api_20.csv', 'api_18.csv', 'api_24.csv', 'api_30.csv', 'api_31.csv', 'api_25.csv', 'api_19.csv', 'api_33.csv', 'api_27.csv', 'api_26.csv', 'api_32.csv', 'api_7.csv', 'api_6.csv', 'api_4.csv', 'api_5.csv', 'api_1.csv', 'api_2.csv', 'api_3.csv']
33


,Place,DogName,Box,Weight,StartPrice,PIR,Split,RunTime,RaceId,Distance,RaceGrade,Track,date,TrainerLName,TrainerFName
0,1.0,swift getaway,1.0,28.1,5.0,331,2.23,18.78,1002606719,320,Maiden,richmond,2024-03-04,poo,poo
1,6.0,gallipoli gal,2.0,27.2,3.6,556,2.29,19.12,1002606719,320,Maiden,richmond,2024-03-04,poo,poo
2,8.0,what kept you,3.0,23.7,8.0,218,2.21,19.42,1002606719,320,Maiden,richmond,2024-03-04,poo,poo
3,5.0,russo,4.0,28.1,6.0,444,2.27,19.10,1002606719,320,Maiden,richmond,2024-03-04,poo,poo
4,3.0,butker,5.0,29.0,6.0,662,2.31,18.86,1002606719,320,Maiden,richmond,2024-03-04,poo,poo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,4.0,northern express,1.0,30.8,6.7,None,NaN,18.49,1003936393,312,5,darwin,2024-03-06,poo,poo
1,5.0,not all there,2.0,23.4,8.6,None,NaN,18.76,1003936393,312,5,darwin,2024-03-06,poo,poo
2,1.0,spiteful miss,4.0,25.7,3.9,0,7.20,18.26,1003936393,312,4,darwin,2024-03-06,poo,poo
3,3.0,solsbury hill,6.0,33.6,5.3,0,NaN,18.47,1003936393,312,5,darwin,2024-03-06,poo,poo


In [10]:
if(missed_run==False):
    missing  = pd.read_csv('api_files/missing.csv')[['missed']]
    missing2 = pd.DataFrame(missed,columns=['missed'])
    missing3 = pd.concat([missing,missing2])
    missing4 = missing3['missed'].unique()
    missing5 = pd.DataFrame(missing4,columns=['missed'])
    missing5.to_csv('api_files/missing.csv',index=False)
else:
    missing5 = pd.DataFrame(missed,columns=['missed'])
    missing5.to_csv('api_files/missing.csv',index=False)
    
missing5

,missed
0,990395989
1,990395990
2,990395991
3,990395992
4,990395993
...,...
69,998755244
70,998755245
71,998755246
72,998764348


In [11]:
files = os.listdir('api_files')
files = [x for x in files if 'missing' not in x and '.DS' not in x]
files.sort()
files
merged = pd.read_csv('api_files/'+files[0])
for file in files[1:]:
    merged=pd.concat([merged,pd.read_csv('api_files/'+file)])

merged=merged.sort_values(by='date')
merged['Track']=np.where(merged['Track'].str.contains('dport @'),'hobart',merged['Track'])
merged

,Place,DogName,Box,Weight,StartPrice,PIR,Split,RunTime,RaceId,Distance,RaceGrade,Track,date,TrainerLName,TrainerFName
0,6.0,chick magnet,1.0,30.5,5.5,566.0,NaN,19.700,990412069,336,5,muswellbrook,2024-01-28,poo,poo
1331,1.0,monkey's style,7.0,34.5,4.9,111.0,5.18000,24.282,992122290,435,5,sale,2024-01-28,poo,poo
1330,3.0,rumour confirmed,6.0,31.9,38.7,743.0,5.44250,24.537,992122290,435,5,sale,2024-01-28,poo,poo
1329,8.0,crew,5.0,34.5,57.1,678.0,5.40750,25.242,992122290,435,5,sale,2024-01-28,poo,poo
1328,7.0,persuasion,4.0,27.1,20.2,887.0,5.47750,25.181,992122290,435,5,sale,2024-01-28,poo,poo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4960,NaN,hartney,NaN,NaN,0.0,NaN,NaN,NaN,1003096223,410,5,horsham,2024-03-08,poo,poo
4961,1.0,wrong bias,1.0,28.9,1.6,11.0,10.20000,23.070,1003096223,410,4,horsham,2024-03-08,poo,poo
4962,2.0,hidden stones,2.0,30.6,2.5,22.0,10.27000,23.135,1003096223,410,4,horsham,2024-03-08,poo,poo
4964,6.0,miss verdansky,4.0,28.5,52.5,76.0,11.05925,23.813,1003096223,410,5,horsham,2024-03-08,poo,poo


In [12]:
a = pd.read_csv('../allRaces2.csv')
b = merged.copy()
c = pd.concat([a,b])
#c = c.drop(columns=['Unnamed: 0.1'])
c = c.drop_duplicates(keep='first')

c['Track']=np.where(c['Track'].str.contains('murray'),'murray bridge',c['Track'])
c['Track']=np.where(c['Track'].str.contains('(ris)'),'richmond straight',c['Track'])
c['Track']=np.where(c['Track'].str.contains('(mep)'),'the meadows',c['Track'])
c['Track']=np.where(c['Track'].str.contains('(sap)'),'sandown park',c['Track'])



c.to_csv('../allRaces1.csv',index=False)
c

/var/folders/_5/kty37fps3g723664y0dwx0_80000gn/T/ipykernel_13041/2967691039.py:8: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  c['Track']=np.where(c['Track'].str.contains('(ris)'),'richmond straight',c['Track'])
/var/folders/_5/kty37fps3g723664y0dwx0_80000gn/T/ipykernel_13041/2967691039.py:9: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  c['Track']=np.where(c['Track'].str.contains('(mep)'),'the meadows',c['Track'])
/var/folders/_5/kty37fps3g723664y0dwx0_80000gn/T/ipykernel_13041/2967691039.py:10: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  c['Track']=np.where(c['Track'].str.contains('(sap)'),'sandown park',c['Track'])


,Place,DogName,Box,Weight,StartPrice,PIR,Split,RunTime,RaceId,Distance,RaceGrade,Track,date,TrainerLName,TrainerFName
0,1.0,raine allen,1.0,27.4,2.4,111,6.68000,28.660,335811282,500,Restricted Win,bendigo,2018-07-01,Grenfell,C
1,2.0,surf a lot,2.0,32.8,6.3,332,7.01000,28.820,335811282,500,Restricted Win,bendigo,2018-07-01,Tyley,C
2,3.0,pingin' bee,6.0,25.5,9.3,443,7.24000,28.930,335811282,500,Restricted Win,bendigo,2018-07-01,Dapiran,P
3,4.0,lucas the great,7.0,32.2,9.1,655,7.31000,29.030,335811282,500,Restricted Win,bendigo,2018-07-01,Hamilton,E
4,5.0,quavo,4.0,28.9,3.4,766,7.38000,29.050,335811282,500,Restricted Win,bendigo,2018-07-01,Dapiran,P
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4960,NaN,hartney,NaN,NaN,0.0,NaN,NaN,NaN,1003096223,410,5,horsham,2024-03-08,poo,poo
4961,1.0,wrong bias,1.0,28.9,1.6,11.0,10.20000,23.070,1003096223,410,4,horsham,2024-03-08,poo,poo
4962,2.0,hidden stones,2.0,30.6,2.5,22.0,10.27000,23.135,1003096223,410,4,horsham,2024-03-08,poo,poo
4964,6.0,miss verdansky,4.0,28.5,52.5,76.0,11.05925,23.813,1003096223,410,5,horsham,2024-03-08,poo,poo


In [13]:
tks = c['Track'].unique()
tks.sort()
tks

array(['addington', 'albion park', 'angle park', 'armidale', 'ascot park',
       'ballarat', 'bathurst', 'bendigo', 'broken hill', 'bulli',
       'bundaberg', 'cairns', 'cambridge', 'canberra', 'cannington',
       'capalaba', 'casino', 'central park', 'clonmel', 'coonabarabran',
       'coonamble', 'cowra', 'cranbourne', 'crayford', 'curraheen park',
       'dapto', 'darwin', 'devonport', 'doncaster', 'drumbo park',
       'dubbo', 'dundalk', 'enniscorthy', 'galway', 'gawler', 'geelong',
       'gosford', 'goulburn', 'grafton', 'gunnedah', 'harlow', 'hatrick',
       'healesville', 'henlow', 'hobart', 'horsham', 'hove', 'ipswich',
       'kempsey', 'kilkenny', 'kinsley', 'launceston', 'lifford',
       'limerick', 'lismore', 'lithgow', 'maitland', 'manawatu',
       'mandurah', 'manukau', 'monmore', 'moree', 'mount gambier',
       'mudgee', 'mullingar', 'murray bridge', 'muswellbrook',
       'newbridge', 'newcastle', 'northam', 'nottingham', 'nowra',
       'oxford', 'pelaw grange

In [14]:
## still missing 
for rID in missing5['missed']:
    url = f'https://topaz.grv.org.au/api/race/{rID}/runs'
    try:
        race = requests.get(url.format(rID),headers=headers).json()[0]
        print(race['track'],race['meetingDate'],race['distance'])
    except JSONDecodeError as e:
        print(rID,'fucked')
        
# race

Capalaba 2024-01-28T00:00:00.000Z 366
Capalaba 2024-01-28T00:00:00.000Z 366
Capalaba 2024-01-28T00:00:00.000Z 366
Capalaba 2024-01-28T00:00:00.000Z 366
Capalaba 2024-01-28T00:00:00.000Z 366
Capalaba 2024-01-28T00:00:00.000Z 366
Capalaba 2024-01-28T00:00:00.000Z 366
Capalaba 2024-01-28T00:00:00.000Z 366
Capalaba 2024-01-28T00:00:00.000Z 366
Capalaba 2024-01-28T00:00:00.000Z 366
Capalaba 2024-01-28T00:00:00.000Z 366
Townsville 2024-01-30T00:00:00.000Z 380
Townsville 2024-01-30T00:00:00.000Z 380
Townsville 2024-01-30T00:00:00.000Z 380
Townsville 2024-01-30T00:00:00.000Z 380
Townsville 2024-01-30T00:00:00.000Z 380
Townsville 2024-01-30T00:00:00.000Z 380
Townsville 2024-01-30T00:00:00.000Z 380
Townsville 2024-01-30T00:00:00.000Z 498
Townsville 2024-01-30T00:00:00.000Z 380
Townsville 2024-01-30T00:00:00.000Z 380
Townsville 2024-01-30T00:00:00.000Z 380
Townsville 2024-01-30T00:00:00.000Z 380
Murray Bridge (MBR) 2024-02-04T00:00:00.000Z 395
Murray Bridge (MBR) 2024-02-04T00:00:00.000Z 395
Murr

In [15]:
big1

,Place,DogName,Box,Weight,StartPrice,PIR,Split,RunTime,RaceId,Distance,RaceGrade,Track,date,TrainerLName,TrainerFName
0,1.0,swift getaway,1.0,28.1,5.0,331,2.23,18.78,1002606719,320,Maiden,richmond,2024-03-04,poo,poo
1,6.0,gallipoli gal,2.0,27.2,3.6,556,2.29,19.12,1002606719,320,Maiden,richmond,2024-03-04,poo,poo
2,8.0,what kept you,3.0,23.7,8.0,218,2.21,19.42,1002606719,320,Maiden,richmond,2024-03-04,poo,poo
3,5.0,russo,4.0,28.1,6.0,444,2.27,19.10,1002606719,320,Maiden,richmond,2024-03-04,poo,poo
4,3.0,butker,5.0,29.0,6.0,662,2.31,18.86,1002606719,320,Maiden,richmond,2024-03-04,poo,poo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,4.0,northern express,1.0,30.8,6.7,None,NaN,18.49,1003936393,312,5,darwin,2024-03-06,poo,poo
1,5.0,not all there,2.0,23.4,8.6,None,NaN,18.76,1003936393,312,5,darwin,2024-03-06,poo,poo
2,1.0,spiteful miss,4.0,25.7,3.9,0,7.20,18.26,1003936393,312,4,darwin,2024-03-06,poo,poo
3,3.0,solsbury hill,6.0,33.6,5.3,0,NaN,18.47,1003936393,312,5,darwin,2024-03-06,poo,poo
